In [1]:
import btk
from ezc3d import c3d
import pandas as pd
import numpy as np
import glob

In [2]:
def cropp_c3dfile(eventsFrame, filename, destiny, no):
    reader = btk.btkAcquisitionFileReader()
    reader.SetFilename(filename)
    reader.Update()
    acq = reader.GetOutput()
 
    writer = btk.btkAcquisitionFileWriter()
    
    for i in range(0, len(eventsFrame)):
        clone = acq.Clone();
        clone.ResizeFrameNumberFromEnd(acq.GetLastFrame() - eventsFrame[i][0] + 1)
        clone.ResizeFrameNumber(eventsFrame[i][1] - eventsFrame[i][0] + 1)
        clone.SetFirstFrame(eventsFrame[i][0])
        clone.ClearEvents()
        for e in btk.Iterate(acq.GetEvents()):
            if ((e.GetFrame() > clone.GetFirstFrame()) and (e.GetFrame() < clone.GetLastFrame())):
                clone.AppendEvent(e)
        clone.SetFirstFrame(1)
        writer.SetInput(clone)
        writer.SetFilename(destiny + '\\' + (filename.split('\\')[-1]).split('.')[0] + '_' + str(no) +'.c3d')
        writer.Update()

In [3]:
path = f'medical_dataset_AK'

In [4]:
filelist =[]
for file in glob.glob(f'{path}\\**\\*.c3d',recursive = True):
    filelist.append(file)

In [5]:
def read_labels(sample):
    e_label = pd.DataFrame(sample['parameters']['EVENT']['LABELS']['value'])
    e_contexts = pd.DataFrame(sample['parameters']['EVENT']['CONTEXTS']['value'])

    times = sample['parameters']['EVENT']['TIMES']['value']
    e_frames = pd.DataFrame(times[1] * 100).astype(int)

    event = pd.concat([e_label, e_contexts, e_frames], axis=1)
    event.columns = ['label', 'context', 'frames']
    event = event.set_index('frames')
    event = event.sort_index(axis=0)

    return event

<h1>One cycle method

In [6]:
def get_one_step_time(sample, label, context, i):
    df = read_labels(sample)
    temp = [frame for frame in df.index if df['label'][frame] == label and df['context'][frame] == context]

    return temp[i], temp[i+1]

In [7]:
def save_cycle(file, site, path, i):
    start, stop = get_one_step_time(sample, 'Foot Strike', site, i)
    eventsFrame = [[int(start), int(stop)]]
    cropp_c3dfile(eventsFrame, file, path, i)

In [8]:
print("Pliki z jednym lub dwoma cyklami chodu:")
for file in filelist:
    sample = c3d(file)
    i=0
    while i < 3:
        try:     
            #First right step
            save_cycle(file, 'Right', 'cut_one_gait_cycle_right', i)    
            #First left step
            save_cycle(file, 'Left', 'cut_one_gait_cycle_left', i)   
        except:
            print(file)
        i+=1

Pliki z jednym lub dwoma cyklami chodu:
medical_dataset_AK\degeneration\degeneration_03.c3d
medical_dataset_AK\degeneration\degeneration_04.c3d
medical_dataset_AK\degeneration\degeneration_04.c3d
medical_dataset_AK\degeneration\degeneration_12.c3d
medical_dataset_AK\healthy\healthy_02.c3d
medical_dataset_AK\healthy\healthy_08.c3d
medical_dataset_AK\healthy\healthy_09.c3d
medical_dataset_AK\healthy\healthy_10.c3d
medical_dataset_AK\healthy\healthy_10.c3d
medical_dataset_AK\healthy\healthy_12.c3d
medical_dataset_AK\healthy\healthy_13.c3d
medical_dataset_AK\healthy\healthy_14.c3d
medical_dataset_AK\healthy\healthy_15.c3d


<h1>Two cycles method

In [6]:
def get_one_step_time_two_cycles(sample, label, context):
    df = read_labels(sample)
    temp = [frame for frame in df.index if df['label'][frame] == label and df['context'][frame] == context]

    return temp[0], temp[2]

In [7]:
def save_two_cycles(file, site, path):
    start, stop = get_one_step_time_two_cycles(sample, 'Foot Strike', site)
    eventsFrame = [[int(start), int(stop)]]
    cropp_c3dfile(eventsFrame, file, path, 0)

In [8]:
for file in filelist:
    sample = c3d(file)
    try:    
        #right
        save_two_cycles(file, 'Right', 'cut_one_gait_two_cycle_right')          
        #left
        save_two_cycles(file, 'Left', 'cut_one_gait_two_cycle_left')         
    except:
        print(file)

medical_dataset_AK\degeneration\degeneration_04.c3d
medical_dataset_AK\healthy\healthy_10.c3d


<h2>Avg

In [9]:
# avg = 132
avg = 276

In [10]:
marker_list = ['LANK', 'RANK', 'LKNE', 'RKNE', 'LFIN', 'RFIN', 'LSHO', 'RSHO', 'LASI', 'RFHD','RWRA','RHEE','LHEE','CentreOfMass']

In [11]:
# path_r = f'cut_one_gait_cycle_right'
# path_l = f'cut_one_gait_cycle_left'
path_r = f'cut_one_gait_two_cycle_right'
path_l = f'cut_one_gait_two_cycle_left'

filelist_r =[]
filelist_l =[]

for file in glob.glob(f'{path_r}\\*.c3d',recursive = True):
    filelist_r.append(file)
    
for file in glob.glob(f'{path_l}\\*.c3d',recursive = True):
    filelist_l.append(file)

In [12]:
filelist_r[0]

'cut_one_gait_two_cycle_right\\degeneration_01_0.c3d'

In [13]:
from pyomeca import Markers

In [14]:
def data_markers(data_path, marker_list, avg):
    data_markers = Markers.from_c3d(data_path, usecols=[marker_list[0]])
    data_markers = data_markers.meca.time_normalize(n_frames=avg)
    data_markers = data_markers.meca.to_wide_dataframe()
    for i in range(len(marker_list)-1):
        
        tmp_markers = Markers.from_c3d(data_path, usecols=[marker_list[i+1]])
        tmp_markers = tmp_markers.meca.time_normalize(n_frames=avg)  
        tmp_markers = tmp_markers.meca.to_wide_dataframe()
        data_markers = data_markers.join(tmp_markers)

        
    cols = [c for c in data_markers.columns if c.lower()[:4] != 'ones']
    markers_dataframe = data_markers[cols]
    
    return markers_dataframe

In [15]:
dm = data_markers(filelist_r[0], marker_list,avg)
dat = np.array(dm['x_LASI'])
dat

array([81.748909  , 81.64160692, 81.46849196, 81.22000752, 80.88693634,
       80.46071222, 79.93371088, 79.29958632, 78.55354961, 77.69258229,
       76.71557811, 75.62338531, 74.41874675, 73.1061772 , 71.69180403,
       70.18314889, 68.58891283, 66.91881173, 65.1833634 , 63.39374509,
       61.56166812, 59.69923444, 57.8186441 , 55.93359694, 54.05682908,
       52.20083826, 50.37780093, 48.59931172, 46.87606438, 45.21754008,
       43.63167295, 42.1245831 , 40.70036725, 39.36100082, 38.10633003,
       36.93419668, 35.84070593, 34.82056979, 33.86753854, 32.97486908,
       32.13577492, 31.34387307, 30.59350237, 29.88098079, 29.2020018 ,
       28.55370442, 27.93463348, 27.3440254 , 26.78154576, 26.24704598,
       25.74035436, 25.26113695, 24.8088237 , 24.38260606, 23.98148768,
       23.6043705 , 23.25015698, 22.9178428 , 22.60659846, 22.31581289,
       22.04511552, 21.79441501, 21.5639973 , 21.3546599 , 21.16844661,
       21.00774623, 20.87540024, 20.77564846, 20.71365884, 20.69

In [16]:
def load_group(filenames):
    
    loaded = []
    
    for file in filenames:
        try:
            result = data_markers(file, marker_list,avg)
            loaded.append(result)
        except:
            print(file)

    return loaded

In [17]:
group = load_group(filelist)

medical_dataset_AK\degeneration\degeneration_09.c3d


In [18]:
x = np.stack(group)
x.shape

(29, 132, 42)

In [19]:
print('(samples, timesteps, features) -> ', x.shape)

(samples, timesteps, features) ->  (29, 132, 42)


In [16]:
import h5py

In [17]:
def create_h5_basic(path, x):
    f = h5py.File(path, mode='w')
    f.create_dataset("x", data=x)
    f.close()

In [18]:
def create_h5(path, right, left):
    f = h5py.File(path, mode='w')
    f.create_dataset("right", data=right)
    f.create_dataset("left", data=left)
    f.close()

# JRD

In [19]:
import math

In [20]:
import numpy as np

def jrd_method(markers):
    
    #arrays with data
    x_lank = np.array(markers['x_LANK'])
    y_lank = np.array(markers['y_LANK'])
    z_lank = np.array(markers['z_LANK'])
    
    x_rank = np.array(markers['x_RANK'])
    y_rank = np.array(markers['y_RANK'])
    z_rank = np.array(markers['z_RANK'])

    #auxiliary tables
    arr_distance_jrd = []
    arr_left = []
    arr_right = []
    time = []
    i = 0
    x_2 = 0
    y_2 = 0
    z_2 = 0

    while i < avg:
    
        x_2 = (x_lank[i] - x_rank[i]) ** 2
        y_2 = (y_lank[i] - y_rank[i]) ** 2
        z_2 = (z_lank[i] - z_rank[i]) ** 2
        distance = math.sqrt(x_2+y_2+z_2)
        
        time.append(i)
        arr_left.append(z_lank[i])
        arr_right.append(z_rank[i])
        arr_distance_jrd.append(distance)
    
        i += 1
        
    return arr_distance_jrd

# JRA

In [21]:
import numpy as np

def jra_method(markers):
    
    #arrays with data
    x_lasi = np.array(markers['x_LASI'])
    y_lasi = np.array(markers['y_LASI'])
    z_lasi = np.array(markers['z_LASI'])
    
    x_lkne = np.array(markers['x_LKNE'])
    y_lkne = np.array(markers['y_LKNE'])
    z_lkne = np.array(markers['z_LKNE'])
    
    x_lank = np.array(markers['x_LANK'])
    y_lank = np.array(markers['y_LANK'])
    z_lank = np.array(markers['z_LANK'])
    
    #auxiliary tables
    first_dist = []
    second_dist = []
    distance = []
    angle = []
    time = []
    
    i = 0
    xl = yl = zl = 0
    xl_1 = yl_1 = zl_1 = 0

    while i < avg:
        
        xl = (x_lasi[i] - x_lkne[i])
        yl = (y_lasi[i] - y_lkne[i])
        zl = (z_lasi[i] - z_lkne[i])
        
        xl_1 = (x_lkne[i] - x_lank[i])
        yl_1 = (y_lkne[i] - y_lank[i])
        zl_1 = (z_lkne[i] - z_lank[i])
        
        time.append(i)
    
        first_dist.append(math.sqrt((xl ** 2) + (yl ** 2) + (zl ** 2)))
        second_dist.append(math.sqrt((xl_1 ** 2) + (yl_1 ** 2) + (zl_1 ** 2)))                   
        distance.append(xl * xl_1 + yl * yl_1 + zl * zl_1)
    
        angle.append(math.acos(distance[i] / (first_dist[i] * second_dist[i])))
        
        i += 1
        
    return angle

# HDF

In [22]:
import numpy as np

from scipy.stats import skew 
def hdf_method(markers):

    #arrays with data
    x_lank = np.array(markers['x_LANK'])
    x_rank = np.array(markers['x_RANK'])
    x_lkne = np.array(markers['x_LKNE'])
    x_rkne = np.array(markers['x_RKNE'])
    x_lfin = np.array(markers['x_LFIN'])
    x_rfin = np.array(markers['x_RFIN'])
    x_lsho = np.array(markers['x_LSHO'])
    x_rsho = np.array(markers['x_RSHO'])
    
    #auxiliaty tables
    #hd1 ankles
    hd1 = []
    hd1_x = 0
    #hd2 knees
    hd2 = []
    hd2_x = 0
    #hd3 wrists
    hd3 = []
    hd3_x = 0
    #hd4 shoulders
    hd4 = []
    hd4_x = 0
    
    #variables
    meanH = []
    stdH = []
    skewH = []
    HDF = [] 
    hdf = []
    time = []
    i = 0
    
    while i < avg:
    
        hd1_x = (x_lank[i] - x_rank[i]) ** 2
        hd1.append(math.sqrt(hd1_x))
        
        hd2_x = (x_lkne[i] - x_rkne[i]) ** 2
        hd2.append(math.sqrt(hd2_x))
        
        hd3_x = (x_lfin[i] - x_rfin[i]) ** 2
        hd3.append(math.sqrt(hd3_x))
        
        hd4_x = (x_lsho[i] - x_rsho[i]) ** 2
        hd4.append(math.sqrt(hd4_x))
        
        time.append(i)
        hdf = [hd1[i], hd2[i], hd3[i], hd4[i]]
        
        meanH.append(np.mean(hdf))
        stdH.append(np.std(hdf))
        skewH.append(skew(hdf))        
        HDF.append([meanH, stdH, skewH])
        
        i += 1
        
    return HDF[0][0], HDF[0][1], HDF[0][2]

# VDF

In [23]:
import numpy as np

def vdf_method(markers):
    
    #arrays with data
    y_rfhd = np.array(markers['y_RFHD'])
    y_rwra = np.array(markers['y_RWRA'])
    y_rsho = np.array(markers['y_RSHO'])
    y_rank = np.array(markers['y_RANK'])
    y_lank = np.array(markers['y_LANK'])
    y_rhee = np.array(markers['y_RHEE'])
    y_lhee = np.array(markers['y_LHEE'])
    y_centreOfMass = np.array(markers['y_CentreOfMass'])
    
    #auxiliary tables
    #vd1 HEIGHT
    vd1 = []
    vd1_y = 0
    #vd2 WRIST RIGHT
    vd2 = []
    vd2_y = 0
    #vd3 SHOULDER RIGHT
    vd3 = []
    vd3_y = 0
    #vd4 ANKLE RIGHT
    vd4 = []
    vd4_y = 0
    #vd5 ANKLE LEFT
    vd5 = []
    vd5_y = 0
    #vd6 DIST. FEET LEFT AND RIGHT
    vd6 = []
    vd6_y = 0
    
    #variables
    meanV = []
    stdV = []
    VDF = [] 
    vdf = []
    time = []
    average = []
    i = 0
    
    while i < avg:
    
        vd1_y = np.array(y_rfhd[i])
        vd1.append(vd1_y)
        
        vd2_y = np.array(y_rwra[i])
        vd2.append(vd2_y)
        
        vd3_y = np.array(y_rsho[i])
        vd3.append(vd3_y)
        
        vd4_y = np.array(y_rank[i])
        vd4.append(vd4_y)
        
        vd5_y = np.array(y_lank[i])
        vd5.append(vd5_y)
        
        vd6_y = np.array((0.5 * (y_rhee[i] - y_lhee[i]) * y_centreOfMass[i]))
        vd6.append(vd6_y)
        
        time.append(i)
        
        average = (vd1_y + vd2_y + vd3_y + vd4_y + vd5_y + vd6_y)
        vdf = [vd1[i], vd2[i], vd3[i], vd4[i], vd5[i], vd6[i]]
        
        meanV.append(np.mean(average))
        stdV.append(np.std(vdf))
        VDF.append([meanV, stdV])
        
        i += 1
        
    return VDF[0][0], VDF[0][1]

# approach

In [24]:
from pyomeca import Markers

In [25]:
def data_features(data_path,avg):
    marker_list = ['LANK', 'RANK', 'LKNE', 'RKNE', 'LFIN', 'RFIN', 'LSHO', 'RSHO', 'LASI', 'RFHD','RWRA','RHEE','LHEE','CentreOfMass']
    markers = data_markers(data_path, marker_list, avg)
    jrd = jrd_method(markers)
    jra = jra_method(markers)
    meanH, stdH, skewH = hdf_method(markers)
    meanV, stdV = vdf_method(markers)
    
    result = [jrd, jra, meanH, stdH, skewH, meanV, stdV]
    
    return pd.DataFrame(result).T

In [26]:
data = data_features(filelist_r[1],avg)
data

,0,1,2,3,4,5,6
0,392.409989,0.278818,334.303560,154.555822,0.480163,-326439.919890,125959.344649
1,392.603387,0.276688,334.295698,155.135906,0.483352,-325974.937026,125777.612731
2,392.535881,0.274385,334.298319,155.672920,0.486443,-325257.778954,125501.845689
3,392.371610,0.271905,334.301975,156.167890,0.489347,-324463.534568,125197.223459
4,392.228513,0.269260,334.300837,156.619338,0.492025,-323709.205752,124907.299815
...,...,...,...,...,...,...,...
271,426.553525,0.282813,342.315028,163.444140,0.581707,-56317.768023,21690.310422
272,426.858342,0.281231,342.078369,164.065968,0.581708,-55328.768115,21312.668260
273,426.886882,0.279090,341.857241,164.680481,0.581928,-54265.337123,20907.192219
274,426.848035,0.276324,341.663129,165.278271,0.582469,-53157.022926,20484.792175


In [27]:
import config as cfg

In [28]:
final_db_r = []
final_db_l = []
r = []
l = []

for file_r in filelist_r:
    try:
        #right
        db_features_r = data_features(file_r,avg)
        final_db_r.append(db_features_r)
    except:
        print(file_r)
        r.append(file_r)
        
for file_l in filelist_l:
    try:
        #left
        db_features_l = data_features(file_l,avg)
        final_db_l.append(db_features_l)
    except:
        print(file_l)
        l.append(file_l)

cut_one_gait_two_cycle_right\degeneration_12_0.c3d
cut_one_gait_two_cycle_left\healthy_08_0.c3d
cut_one_gait_two_cycle_left\healthy_13_0.c3d


In [29]:
create_h5('features_database_gait_two_cycles_both_legs.h5', final_db_r, final_db_l)

In [30]:
l

['cut_one_gait_two_cycle_left\\healthy_08_0.c3d',
 'cut_one_gait_two_cycle_left\\healthy_13_0.c3d']

In [31]:
r

['cut_one_gait_two_cycle_right\\degeneration_12_0.c3d']